In [1]:
import os
import pickle 
import numpy as np
import pandas as pd
import re
from pprint import pprint

In [2]:
output_dir="../../data/train_sentence_pkl"
start="sentence_"
end=".pkl"
pkl_files = [fname for fname in os.listdir(output_dir) if fname.startswith(start) and fname.endswith(end)]
def natural_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]
pkl_files.sort(key=natural_key)
print(pkl_files)

['sentence_1.pkl', 'sentence_2.pkl', 'sentence_3.pkl', 'sentence_4.pkl', 'sentence_5.pkl', 'sentence_6.pkl', 'sentence_7.pkl', 'sentence_8.pkl', 'sentence_9.pkl', 'sentence_10.pkl', 'sentence_11.pkl', 'sentence_12.pkl', 'sentence_13.pkl', 'sentence_14.pkl', 'sentence_15.pkl', 'sentence_16.pkl', 'sentence_17.pkl', 'sentence_18.pkl']


### 파일 크기

In [3]:
for file_path in sorted(pkl_files):
    size = os.path.getsize(os.path.join(output_dir, file_path))
    print(f"파일 크기: {size} 바이트")

파일 크기: 660154044 바이트
파일 크기: 660140137 바이트
파일 크기: 660331647 바이트
파일 크기: 659210931 바이트
파일 크기: 660175960 바이트
파일 크기: 660049060 바이트
파일 크기: 660248623 바이트
파일 크기: 659647207 바이트
파일 크기: 659922195 바이트
파일 크기: 279403072 바이트
파일 크기: 660831294 바이트
파일 크기: 659023740 바이트
파일 크기: 659922660 바이트
파일 크기: 660756032 바이트
파일 크기: 659245588 바이트
파일 크기: 659906184 바이트
파일 크기: 660442959 바이트
파일 크기: 659990378 바이트


### 저장 형태 확인

In [4]:
with open("../../data/test_sentence_pkl/sentence_1.pkl", "rb") as f:
    sentence_1=pickle.load(f)


In [5]:
sentence_1.head(1)

,title,paragraph_index,sentence_index,sentence_text,sentence_emb,lof
0,공중 도덕의 의의와 필요성,0,0,도덕이란 원래 개인의 자각에서 출발해 자기 의지로써 행동하는 일이다.,"[-0.051884104, -0.5708666, -0.3369897, -0.0271...",1.020817


In [6]:
sentence_1.dtypes

title               object
paragraph_index      int64
sentence_index       int64
sentence_text       object
sentence_emb        object
lof                float64
dtype: object

### 전체 데이터 개수 및 title 연속 여부

In [7]:
data_points=[]
titles=[]
paragraph=[]

nan_list=[]
nan_details = []
df_null = None
for file_name in pkl_files:
    with open(os.path.join(output_dir, file_name), "rb") as f:
        sentence=pickle.load(f)
        # data points
        titles.append(sentence["title"].nunique())
        paragraph.append(sentence.groupby(['title', 'paragraph_index']).ngroups)
        data_points.append(len(sentence))
        # title 연속 여부
        block_titles = sentence['title'][sentence['title'] != sentence['title'].shift()].tolist()
        unique_titles = sentence['title'].unique().tolist()
        if sorted(block_titles) == sorted(unique_titles):
            print(f"{file_name}: ✅ 모든 title 연속 ⭕️")
        else:
            print(f"{file_name}⚠️  모든 title 연속 ❌")
        # 결측치
        nan_bool=sentence.isnull().values.any()
        nan_list.append(nan_bool)
        if nan_bool:
            df_null = sentence[sentence.isnull().any(axis=1)]
            nan_details.append((file_name, df_null))

sentence_1.pkl: ✅ 모든 title 연속 ⭕️
sentence_2.pkl: ✅ 모든 title 연속 ⭕️
sentence_3.pkl: ✅ 모든 title 연속 ⭕️
sentence_4.pkl: ✅ 모든 title 연속 ⭕️
sentence_5.pkl: ✅ 모든 title 연속 ⭕️
sentence_6.pkl: ✅ 모든 title 연속 ⭕️
sentence_7.pkl: ✅ 모든 title 연속 ⭕️
sentence_8.pkl: ✅ 모든 title 연속 ⭕️
sentence_9.pkl: ✅ 모든 title 연속 ⭕️
sentence_10.pkl: ✅ 모든 title 연속 ⭕️
sentence_11.pkl: ✅ 모든 title 연속 ⭕️
sentence_12.pkl: ✅ 모든 title 연속 ⭕️
sentence_13.pkl: ✅ 모든 title 연속 ⭕️
sentence_14.pkl: ✅ 모든 title 연속 ⭕️
sentence_15.pkl: ✅ 모든 title 연속 ⭕️
sentence_16.pkl: ✅ 모든 title 연속 ⭕️
sentence_17.pkl: ✅ 모든 title 연속 ⭕️
sentence_18.pkl: ✅ 모든 title 연속 ⭕️


In [8]:
print(f"title: {sum(titles)}")
print(f"paragraph: {sum(paragraph)}")
print(f"total(sentence): {sum(data_points)}")

title: 97172
paragraph: 1226364
total(sentence): 3484607


### 결측치

In [9]:
if not nan_list:
    print("⚠️ nan_list ❌")
elif any(nan_list):
    print("⚠️ 결측치 존재")
else:
    print("✅ 결측치 없음")

✅ 결측치 없음


In [ ]:
# emb_shape
for title, group in sentence_1.groupby('title', sort=False):
    # print(f"title: {title}")
    # print(f"group: {group}")
    emb_matrix = np.vstack(group['sentence_emb'].values)
    # print(emb_matrix)
    print(f"emb_matrix.shape: {emb_matrix.shape}")

    break